In [1]:
import os
from dotenv import load_dotenv
from faker import Faker
import random
import datetime

fake = Faker()
# Le .env est aussi à la racine
load_dotenv(".env")

USER_PSQL = os.environ.get("POSTGRESQL_LOCAL_USER")
PASSWORD_PSQL = os.environ.get("POSTGRESQL_LOCAL_PASSWORD")

import psycopg2
conn = psycopg2.connect(dbname="app_auth", user=USER_PSQL, password=PASSWORD_PSQL, host="localhost", port="5432")

In [2]:
cur = conn.cursor()

cur.execute("""
DROP TABLE IF EXISTS user_for_comparison;            
            """)
cur.execute("""
CREATE TABLE user_for_comparison (
    user_id SERIAL PRIMARY KEY,
    firstname VARCHAR(50),
    lastname VARCHAR(50),
    email VARCHAR(100),
    username VARCHAR(50),
    password VARCHAR(100),
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
""")
cur.execute("""
ALTER TABLE user_for_comparison
ADD CONSTRAINT username_length CHECK (LENGTH(username) >= 8);

ALTER TABLE user_for_comparison
ADD CONSTRAINT email_unique UNIQUE(email);

ALTER TABLE user_for_comparison
ADD CONSTRAINT password_length CHECK (LENGTH(password) >= 8);
""")


In [3]:
for _ in range(100000):
    firstname = fake.first_name()
    lastname = fake.last_name()
    email = fake.unique.email()
    username = fake.user_name() + str(random.randint(100, 999))  
    password = fake.password(length=random.randint(8, 20))
    created_at = fake.date_time_between_dates(datetime_start=datetime.datetime(2023, 1, 1),
                                              datetime_end=datetime.datetime(2023, 12, 31))
    while len(username) < 8:
            username = fake.user_name() + str(random.randint(100, 999))
    cur.execute("""
    INSERT INTO user_for_comparison (firstname, lastname, email, username, password, created_at)
    VALUES (%s, %s, %s, %s, %s, %s);
    """, (firstname, lastname, email, username, password, created_at))


conn.commit()



In [4]:
usernames_to_query = ["elizabethjohnson950", "mrobinson543", "lcantu498","pamelawilkins954"] 


for username in usernames_to_query:
    query = f"SELECT * FROM user_for_comparison WHERE username = '{username}'"
    
    print(f"Temps d'exécution pour {username}:")
    %timeit -r 1 -o cur.execute(query) 


    results = cur.fetchall()


Temps d'exécution pour elizabethjohnson950:
7.17 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 100 loops each)
Temps d'exécution pour mrobinson543:
7.67 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 100 loops each)
Temps d'exécution pour lcantu498:
8.88 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 100 loops each)
Temps d'exécution pour pamelawilkins954:
8.48 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 100 loops each)


In [5]:

cur.execute("CREATE INDEX idx_username ON user_for_comparison (username);")

usernames_to_query = ["elizabethjohnson950", "mrobinson543", "lcantu498","pamelawilkins954"] 


for username in usernames_to_query:
    query = f"SELECT * FROM user_for_comparison WHERE username = '{username}'"
    
    print(f"Temps d'exécution pour {username}:")
    %timeit -r 1 -o cur.execute(query) 


    results = cur.fetchall()

cur.close()
conn.close()

Temps d'exécution pour elizabethjohnson950:
56.4 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 10,000 loops each)
Temps d'exécution pour mrobinson543:
54 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 10,000 loops each)
Temps d'exécution pour lcantu498:
54.8 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 10,000 loops each)
Temps d'exécution pour pamelawilkins954:
54.4 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 10,000 loops each)
